## Getting Started

In [2]:
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.geometry import Envelope
from arcgis.features.manage_data import generate_tessellation
from arcgis.features import FeatureSet
from getpass import getpass

In [3]:
password=getpass()
gis = GIS(username="YOUR-USERNAME",password=password)

········


## Build Analysis Areas

In [4]:
inland_empire_map = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map.basemap = 'openstreetmap'
inland_empire_map

MapView(layout=Layout(height='400px', width='100%'))

In [5]:
hexagons = generate_tessellation(None, bin_size=1, bin_size_unit='SquareMiles', bin_type='HEXAGON', context={"extent":inland_empire_map.extent})

In [6]:
inland_empire_map.add_layer(hexagons)

In [7]:
geometry_areas = [{"geometry":f["geometry"].copy()} for f in hexagons.layer.featureSet.features]

for area in geometry_areas:
    area["geometry"]["spatialReference"]={"wkid":3857}

In [8]:
geometry_areas[0]

{'geometry': {'rings': [[[-13062306.6791, 4028583.2685000002],
    [-13062906.5407, 4027535.6648999974],
    [-13064109.4833, 4027533.6699],
    [-13064712.784400001, 4028579.277800001],
    [-13064113.0335, 4029627.0714000016],
    [-13062909.8707, 4029629.0671000034],
    [-13062306.6791, 4028583.2685000002]]],
  'spatialReference': {'wkid': 3857}}}

In [9]:
enriched_areas=enrich(study_areas=geometry_areas, analysis_variables=["KeyGlobalFacts.TOTPOP","AtRisk.SENIORS_CY"])

In [10]:
enriched_areas

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE
0,0,1,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,2901,209,"{""rings"": [[[-117.34069735692039, 33.998370815..."
1,1,2,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,1109,94,"{""rings"": [[[-117.3244995205204, 34.0061859289..."
2,2,3,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,2,0,"{""rings"": [[[-117.30827571292721, 33.998408265..."
3,3,4,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,1640,316,"{""rings"": [[[-117.29207489771376, 34.006219103..."
4,4,5,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,32,6,"{""rings"": [[[-117.27585404467956, 33.998437158..."
...,...,...,...,...,...,...,...,...,...,...
71,71,72,US,BlockApportionment:US.BlockGroups,2.191,2.576,0,0,0,"{""rings"": [[[-117.06495161840454, 34.115345134..."
72,72,73,US,BlockApportionment:US.BlockGroups,2.191,2.576,0,0,0,"{""rings"": [[[-117.04873077794674, 34.107533359..."
73,73,74,US,BlockApportionment:US.BlockGroups,2.191,2.576,0,0,0,"{""rings"": [[[-117.03248542156655, 34.115309802..."
74,74,75,US,BlockApportionment:US.BlockGroups,2.191,2.576,0,0,0,"{""rings"": [[[-117.01626756800707, 34.107493746..."


In [11]:
inland_empire_map2 = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map2.basemap = 'openstreetmap'
inland_empire_map2

MapView(layout=Layout(height='400px', width='100%'))

In [12]:
enriched_areas["SENIORS_PERCENT"] = enriched_areas["SENIORS_CY"]  / enriched_areas["TOTPOP"] * 100

In [13]:
senior_populated_area = enriched_areas[enriched_areas["SENIORS_PERCENT"] > 25]
senior_populated_area.head()

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE,SENIORS_PERCENT
33,33,34,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,552,155,"{""rings"": [[[-117.16237255271459, 34.021856845...",28.079710
34,34,35,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,444,120,"{""rings"": [[[-117.14616058491686, 34.014057803...",27.027027
35,35,36,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,1467,386,"{""rings"": [[[-117.12994195072142, 34.021847229...",26.312202
36,36,37,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,511,132,"{""rings"": [[[-117.11373295455066, 34.014043908...",25.831703
53,53,54,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,2197,779,"{""rings"": [[[-117.19480539330945, 34.037448478...",35.457442


In [14]:
inland_empire_map2.add_layer(FeatureSet.from_dataframe(senior_populated_area.copy()))

In [15]:
inland_empire_map2.add_layer(FeatureSet.from_dataframe(enriched_areas), {'renderer':'ClassedSizeRenderer','field_name':'SENIORS_PERCENT', 'opacity':0.5})